In [44]:
import numpy
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error



df = pd.read_csv('Dataset/bitstampUSD_1-min_data_2012-01-01_to_2020-04-22.csv')
df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# fix random seed for reproducibility
numpy.random.seed(7)

#Selecting one country for study
df = df[["Timestamp", "Close", "Volume_(BTC)"]]
print(df.shape)
#sles.set_index("Timestamp", inplace=True)
nan_value = float("NaN")
df.dropna(subset = ["Close"], inplace=True)
print(df.shape)
df = df.iloc[(len(df)*4//5 ) : (len(df)), :]
#print(df)
print(df.shape)
dataset = df.values
dataset = dataset.astype('float32')

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)


# split into train and test sets
train_size = int(len(dataset) * 0.75)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

(4363457, 3)
(3126480, 3)
(625296, 3)
468972 156324


In [ ]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Epoch 1/100
 - 1202s - loss: 8.3700e-05
Epoch 2/100
 - 1048s - loss: 3.2444e-07
Epoch 3/100
 - 1010s - loss: 3.2085e-07
Epoch 4/100
 - 1519s - loss: 3.0662e-07
Epoch 5/100
 - 920s - loss: 2.8489e-07
Epoch 6/100
 - 962s - loss: 2.5716e-07
Epoch 7/100
 - 1069s - loss: 2.1749e-07
Epoch 8/100
 - 1097s - loss: 1.8081e-07
Epoch 9/100
 - 1054s - loss: 1.6436e-07
Epoch 10/100
 - 993s - loss: 1.4851e-07
Epoch 11/100
 - 888s - loss: 1.3139e-07
Epoch 12/100
 - 842s - loss: 1.2985e-07
Epoch 13/100
 - 792s - loss: 1.2252e-07
Epoch 14/100
 - 1944s - loss: 1.2097e-07
Epoch 15/100
 - 804s - loss: 1.1837e-07
Epoch 16/100
 - 822s - loss: 1.1880e-07
Epoch 17/100
 - 798s - loss: 1.1523e-07
Epoch 18/100
 - 839s - loss: 1.1586e-07
Epoch 19/100
 - 56914s - loss: 1.1237e-07
Epoch 20/100
 - 1203s - loss: 1.1707e-07
Epoch 21/100
 - 999s - loss: 1.1595e-07
Epoch 22/100
 - 1087s - loss: 1.1203e-07
Epoch 23/100
 - 1161s - loss: 1.1169e-07
Epoch 24/100
 - 1135s - loss: 1.1321e-07
Epoch 25/100
 - 899s - loss: 1.1204